# Name: Sakshi Deokar

# Library: Pytables

URL: https://www.pytables.org/usersguide/tutorials.html


# Description: 
PyTables is a Python library that efficiently manages large datasets using the HDF5 format. It optimizes storage space, speeds up data access, and supports compression, making it ideal for handling complex data structures and large volumes of information in various fields, including science, engineering, finance, and analytics.

In [3]:
!pip install tables

### Installation

Install PyTables using pip to manage HDF5 file formats efficiently.

### Import Libraries

Import necessary Python libraries for handling data and working with HDF5 files.

In [4]:
# Import PyTables for handling HDF5 files and pandas for data manipulation
import tables as tb
import pandas as pd
import numpy as np

### Data Definition and HDF5 File Creation

Define a data structure for storing sales data and create an HDF5 file to hold this data. This step involves:
- Defining a Python class to structure the sales data.
- Creating an HDF5 file and a table within it for data storage.

In [5]:
# Define a class to represent sales data
class Sales(tb.IsDescription):
    transaction_date = tb.StringCol(10)  # Date of transaction (formatted as YYYY-MM-DD)
    product_id = tb.StringCol(10)        # Product identifier
    quantity_sold = tb.IntCol()          # Quantity of product sold
    sales_amount = tb.FloatCol()         # Total sales amount for the transaction

# Create an HDF5 file and a table within it to store sales data
h5file = tb.open_file("sales_data.h5", mode="w", title="Sales Data")
sales_table = h5file.create_table("/", "sales", Sales, "Sales Data Table")


### Data Insertion and Flushing to Disk

Insert sample sales data into the HDF5 table and flush the data to ensure it's written to the file. This involves:
- Adding data to the table row by row.
- Flushing the table's buffer to write changes to the disk.

In [6]:
# To Get a reference to the sales table
sales = h5file.root.sales

# Insert sample sales data into the table
sales_row = sales.row
sales_row['transaction_date'] = '2024-04-17'  # Set transaction date
sales_row['product_id'] = 'P001'              # Set product ID
sales_row['quantity_sold'] = 10               # Set quantity sold
sales_row['sales_amount'] = 100.0             # Set sales amount
sales_row.append()                            # Add the record to the table

# Repeat insertion for another product
sales_row['transaction_date'] = '2024-04-17'
sales_row['product_id'] = 'P002'
sales_row['quantity_sold'] = 5
sales_row['sales_amount'] = 50.0
sales_row.append()

# Flush the data to disk to ensure it is written to the file
sales.flush()


# Calculate Total Sales for a Specific Date

In [12]:
# Define a function to query total sales amount for a given date
def get_total_sales_for_date(transaction_date):
    query = (f"transaction_date == '{transaction_date}'")
    result = sales.read_where(query)
    total_sales = result['sales_amount'].sum()
    return total_sales

# Get total sales amount for a specific date
total_sales = get_total_sales_for_date('2024-04-17')
print("Total Sales for 2024-04-17:", total_sales)

Total Sales for 2024-04-17: 150.0


# Advanced Data Manipulation and Analysis¶
Further develop the data manipulation and analysis capabilities by:

Extending the data model with additional fields.
Generating a larger dataset.
Defining and using a function to calculate rolling averages of sales data based on specific criteria.

In [21]:
# To Define a class to represent sales data with additional fields
class SalesAdvanced(tb.IsDescription):
    transaction_date = tb.StringCol(10)
    store_id = tb.StringCol(10)
    city = tb.StringCol(20)
    product_id = tb.StringCol(10)
    product_category = tb.StringCol(20)
    quantity_sold = tb.IntCol()
    sales_amount = tb.FloatCol()

# Create an HDF5 file and a table within it to store sales data
h5file_advanced = tb.open_file("sales_data_advanced.h5", mode="w", title="Advanced Sales Data")
sales_table_advanced = h5file_advanced.create_table("/", "sales_advanced", SalesAdvanced, "Advanced Sales Data Table")

# Generate sample sales data
np.random.seed(0)
dates = np.random.choice(pd.date_range('2020-01-01', '2023-12-31'), size=10000)
store_ids = np.random.choice(['S001', 'S002', 'S003'], size=10000)
cities = np.random.choice(['New York', 'London', 'Tokyo'], size=10000)
product_ids = np.random.choice(['P001', 'P002', 'P003'], size=10000)
product_categories = np.random.choice(['Electronics', 'Clothing', 'Home Decor'], size=10000)
quantities_sold = np.random.randint(1, 100, size=10000)
sales_amounts = np.random.uniform(10, 1000, size=10000)

# Insert sample sales data into the table
sales_row_advanced = sales_table_advanced.row
for i in range(len(dates)):
    date_str = pd.to_datetime(dates[i]).strftime('%Y-%m-%d')  # Convert and format date
    sales_row_advanced['transaction_date'] = date_str
    sales_row_advanced['store_id'] = store_ids[i]
    sales_row_advanced['city'] = cities[i]
    sales_row_advanced['product_id'] = product_ids[i]
    sales_row_advanced['product_category'] = product_categories[i]
    sales_row_advanced['quantity_sold'] = quantities_sold[i]
    sales_row_advanced['sales_amount'] = sales_amounts[i]
    sales_row_advanced.append()

# Flush the data to disk and close the file
sales_table_advanced.flush()


In [22]:
# Convert NumPy datetime object to Python datetime object before formatting
dates_python = [date.astype('M8[D]').astype('O') for date in dates]


# Insert sample sales data into the table
sales_row_advanced = sales_table_advanced.row
for i in range(len(dates)):
    sales_row_advanced['transaction_date'] = dates_python[i].strftime('%Y-%m-%d')
    sales_row_advanced['store_id'] = store_ids[i]
    sales_row_advanced['city'] = cities[i]
    sales_row_advanced['product_id'] = product_ids[i]
    sales_row_advanced['product_category'] = product_categories[i]
    sales_row_advanced['quantity_sold'] = quantities_sold[i]
    sales_row_advanced['sales_amount'] = sales_amounts[i]
    sales_row_advanced.append()

# Flush the data to disk
sales_table_advanced.flush()
h5file_advanced.close()

In [23]:
#to read and verify that the data was stored correctly
with tb.open_file("sales_data_advanced.h5", mode="r") as file:
    table = file.root.sales_advanced
    data = [row.fetch_all_fields() for row in table.iterrows()]
    print(data[:10])  # Print the first 10 records to verify


[(b'London', b'Home Decor', b'P001', 3, 409.73509426, b'S002', b'2021-11-15'), (b'London', b'Clothing', b'P003', 80, 564.15181473, b'S002', b'2021-07-13'), (b'New York', b'Home Decor', b'P003', 16, 835.35314682, b'S002', b'2023-05-01'), (b'London', b'Home Decor', b'P002', 56, 343.57815543, b'S001', b'2022-04-15'), (b'Tokyo', b'Home Decor', b'P003', 75, 76.69610362, b'S002', b'2022-02-02'), (b'New York', b'Electronics', b'P001', 92, 81.34099112, b'S003', b'2023-10-15'), (b'Tokyo', b'Electronics', b'P002', 34, 344.48207346, b'S003', b'2022-10-30'), (b'London', b'Home Decor', b'P002', 46, 257.93702239, b'S001', b'2020-10-04'), (b'London', b'Electronics', b'P001', 85, 180.24478043, b'S001', b'2021-08-22'), (b'Tokyo', b'Home Decor', b'P001', 50, 286.59070769, b'S001', b'2022-12-30')]
